In [141]:
from datetime import datetime
import math

import ephem
import numpy as np
import palpy

import lsst.sims.skybrightness as sb
from lsst.sims.utils import _altAzPaFromRaDec, ObservationMetaData

In [142]:
# Setup observing site information and the date/time for the test.
date_str = '2016/03/31'
time_str = '04:00:00'
date_string = date_str + " " + time_str
dt = datetime.strptime(date_string, "%Y/%m/%d %H:%M:%S")
mjd = palpy.caldj(dt.year, dt.month, dt.day)
mjd += (dt.hour / 24.0) + (dt.minute / 1440.) + (dt.second / 86400.)
print("MJD: {}".format(mjd))
obs_md = ObservationMetaData(mjd=mjd)

MJD: 57478.1666667


In [143]:
# Use pyephem to get a "truth" altitude and azimuth for Canopus from the LSST Observatory site in
# March since Canopus is above the horizon at night there at this time. I'm using Canopus as a
# surrogate for an OpSim RA, Dec pointing
canopus = ephem.star('Canopus')

cerro_pachon = ephem.Observer()
cerro_pachon.date = date_string
cerro_pachon.lat = str(obs_md.site.latitude)
cerro_pachon.lon = str(obs_md.site.longitude)

canopus.compute(cerro_pachon)
print("Site Latitude/Longitude: ({}, {})".format(cerro_pachon.lat, cerro_pachon.lon))
print("Current Sidereal Time: {}".format(cerro_pachon.sidereal_time()))
# Below shifts to the local time where notebook is being run, not the observing site timezone
#print("Current Local Time: {}".format(ephem.localtime(cerro_pachon.date)))
print("Canopus: (RA={}, Dec={})".format(canopus.ra, canopus.dec))
print("Canopus: (Alt={}, Az={})".format(canopus.alt, canopus.az))
ra_arr = np.array([canopus.ra])
dec_arr = np.array([canopus.dec])

Site Latitude/Longitude: (-30:14:39.8, -70:44:57.8)
Current Sidereal Time: 11:52:51.20
Canopus: (RA=6:24:18.62, Dec=-52:42:47.2)
Canopus: (Alt=28:12:59.2, Az=222:54:36.3)


/home/mareuter/miniconda2/envs/opsim4_py2/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: PyOS_ascii_strtod and PyOS_ascii_atof are deprecated.  Use PyOS_string_to_double instead.
/home/mareuter/miniconda2/envs/opsim4_py2/lib/python2.7/site-packages/ipykernel/__main__.py:8: DeprecationWarning: PyOS_ascii_strtod and PyOS_ascii_atof are deprecated.  Use PyOS_string_to_double instead.
/home/mareuter/miniconda2/envs/opsim4_py2/lib/python2.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: PyOS_ascii_strtod and PyOS_ascii_atof are deprecated.  Use PyOS_string_to_double instead.


In [144]:
# Use methodology for finding altitude and azimuth from OpSim version 3 code.
lst_rad = palpy.gmst(mjd) + math.radians(obs_md.site.longitude)
if lst_rad < 0.:
    lst_rad += 2.0 * math.pi
print("Local Sidereal Time: {}".format(math.degrees(lst_rad) / 15.0))
ha_rad = lst_rad - canopus.ra
az_o3, alt_o3 = palpy.de2h(ha_rad, canopus.dec, math.radians(obs_md.site.latitude))
print("Canopus: (Alt={}, Az={})".format(math.degrees(alt_o3), math.degrees(az_o3)))

Local Sidereal Time: 11.8809428386
Canopus: (Alt=28.1864235178, Az=222.909943309)


In [145]:
# Use the high precision altitude and azimuth finder from sims_utils
alt_hp, az_hp, _ = _altAzPaFromRaDec(ra_arr, dec_arr, obs_md)
print("Canopus: (Alt={}, Az={})".format(np.degrees(alt_hp)[0], np.degrees(az_hp)[0]))

Canopus: (Alt=28.3541880646, Az=222.929920411)


In [146]:
# Use Peter's stupid fast conversion routine for finding altitude and azimuth
alt_sf, az_sf = sb.stupidFast_RaDec2AltAz(ra_arr, dec_arr,
                                          math.radians(obs_md.site.latitude),
                                          math.radians(obs_md.site.longitude),
                                          mjd)
print("Canopus: (Alt={}, Az={})".format(np.degrees(alt_sf)[0], np.degrees(az_sf)[0]))

Canopus: (Alt=28.1864235178, Az=222.909943309)


In [147]:
def calculate_differences(d_alt, d_az, t_alt, t_az):
    delta_alt = math.fabs(d_alt - t_alt)
    delta_az = math.fabs(d_az - t_az)
    sqrt_quad_sum = math.sqrt(delta_az**2 + delta_alt**2)
    return delta_alt, delta_az, sqrt_quad_sum

def print_comps(msg, dalt, daz, sqs):
    print("{}: (Delta Alt={:.3f}, Delta Az={:.3f}, "
          "Sqrt Quad Sum={:.3f})".format(msg, math.degrees(dalt),
                                         math.degrees(daz), math.degrees(sqs)))

In [148]:
# First thing to note is that palpy::de2h and the stupidFast routine produce the same answers. 
# Calculate the differences between "truth" position and the determined positions. 
# Displayed units are degrees.
print_comps("Opsim3        ", *calculate_differences(alt_o3, az_o3, canopus.alt, canopus.az))
print_comps("High Precision", *calculate_differences(alt_hp, az_hp, canopus.alt, canopus.az))
print_comps("StupidFast    ", *calculate_differences(alt_sf, az_sf, canopus.alt, canopus.az))

Opsim3        : (Delta Alt=0.030, Delta Az=0.000, Sqrt Quad Sum=0.030)
High Precision: (Delta Alt=0.138, Delta Az=0.020, Sqrt Quad Sum=0.139)
StupidFast    : (Delta Alt=0.030, Delta Az=0.000, Sqrt Quad Sum=0.030)
